In [1]:
from langchain_core.output_parsers import CommaSeparatedListOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

In [2]:
import os
from dotenv import load_dotenv

load_dotenv(dotenv_path="../../.env")
api_key = os.getenv("API_KEY")

model = ChatOpenAI(
    model='gpt-4.1',
    temperature=0,
    api_key=api_key
)

In [6]:
output_parser = CommaSeparatedListOutputParser()
format_instructions = output_parser.get_format_instructions()

prompt = PromptTemplate(
    template="List five {subject}.\n{format_instructions}",
    input_variables=['subject'],
    partial_variables={'format_instructions' : format_instructions}
)
chain = prompt  |  model  | output_parser
chain.invoke({"subject" : "독산동 맛집"})

['진미통닭', '독산동칼국수', '삼거리포차', '명동칼국수', '황소곱창']

In [5]:
from langchain.output_parsers import ResponseSchema, StructuredOutputParser

In [7]:
response_list = [
    ResponseSchema(name='answer', description="사용자 질문에 대한 답변"),
    ResponseSchema(
        name='source',
        description='사용자의 질문에 답하기 위해 사용된 `출처`, `웹사이트주소` 이어야 한다'
    )
]
output_parser = StructuredOutputParser.from_response_schemas(response_list)

format_instructions = output_parser.get_format_instructions()
prompt = PromptTemplate(
    template="answer the users question as best as possible.\n{format_instructions}\n{question}",
    input_variables=['question'],
    partial_variables={'format_instructions' :format_instructions }
)

chain = prompt  |  model  | output_parser
chain.invoke({"question" : "독산동 맛집은 어디인가요?"})

{'answer': "독산동에서 인기 있는 맛집으로는 '독산동칼국수', '독산동쭈꾸미', '진미식당', '독산동곱창', '명동칼국수' 등이 있습니다. 특히 독산동칼국수는 오랜 전통과 진한 국물로 유명하며, 쭈꾸미 요리 전문점도 매운맛을 좋아하는 분들에게 인기가 많습니다.",
 'source': 'https://www.mangoplate.com/search/%EB%8F%85%EC%82%B0%EB%8F%99'}

In [8]:
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field
from langchain_core.prompts import ChatPromptTemplate

class Topic(BaseModel):
    description : str = Field(description='주제에 대한 간결한 설명'),
    hashtags : str = Field(description="해시태그 형식의 키워드(2개 이상)")

parser = JsonOutputParser(pydantic_object=Topic)
prompt = ChatPromptTemplate(
    [
        ("system", "딩신은 친절한 AI 어시스턴트 입니다. 질문에 간결하게 답변하세요"),
        ("user", "#FORMAT: {format_instructions}\n\n#Question: {question}")
    ]
)
prompt = prompt.partial(format_instructions=parser.get_format_instructions())

text = input("질의 :")
chain.invoke({"question" : text})

/home/user1/miniconda3/envs/openai/lib/python3.11/site-packages/pydantic/json_schema.py:2324: PydanticJsonSchemaWarning: Default value (FieldInfo(annotation=NoneType, required=True, description='주제에 대한 간결한 설명'),) is not JSON serializable; excluding default from JSON schema [non-serializable-default]
  warnings.warn(message, PydanticJsonSchemaWarning)


{'answer': '물론입니다! 사용자 질문을 남겨주시면 최선을 다해 답변드리겠습니다.',
 'source': 'https://openai.com/blog/chatgpt'}

In [13]:
from llama_index.core.readers import SimpleDirectoryReader
documents = SimpleDirectoryReader("llama_data").load_data()

In [14]:
from llama_index.core import VectorStoreIndex, ServiceContext
from llama_index.llms.openai import OpenAI